# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** This is a classification problem because we are trying to determine between two binary outcomes, intervention or no. Therefore, the output variable is discrete and not continuous. While we could view this problem as a spectrum, 0 being not intervention and 1 being intervention, and hence a regression problem, it makes less sense as there isn't a clear action if the model predicts a value of 0.5.  Therefore, I will interpret this as a classification problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
print(student_data.columns)
print(np.max(student_data.age))
print(np.min(student_data.age))
student_data

Index([u'school', u'sex', u'age', u'address', u'famsize', u'Pstatus', u'Medu',
       u'Fedu', u'Mjob', u'Fjob', u'reason', u'guardian', u'traveltime',
       u'studytime', u'failures', u'schoolsup', u'famsup', u'paid',
       u'activities', u'nursery', u'higher', u'internet', u'romantic',
       u'famrel', u'freetime', u'goout', u'Dalc', u'Walc', u'health',
       u'absences', u'passed'],
      dtype='object')
22
15


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes
5,GP,M,16,U,LE3,T,4,3,services,other,...,yes,no,5,4,2,1,2,5,10,yes
6,GP,M,16,U,LE3,T,2,2,other,other,...,yes,no,4,4,4,1,1,3,0,yes
7,GP,F,17,U,GT3,A,4,4,other,teacher,...,no,no,4,1,4,1,1,1,6,no
8,GP,M,15,U,LE3,A,3,2,services,other,...,yes,no,4,2,2,1,1,1,0,yes
9,GP,M,15,U,GT3,T,3,4,other,other,...,yes,no,5,5,1,1,1,5,0,yes


In [3]:
student_data

# TODO: Calculate number of students
n_students = len(student_data.index)

# TODO: Calculate number of features
# Subtract one to account for the label column
n_features = len(student_data.columns)-1

# TODO: Calculate passing students
n_passed = len(student_data.index[student_data.passed=='yes'])

# TODO: Calculate failing students
n_failed = len(student_data.index[student_data.passed=='no'])

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/(float(n_passed+n_failed))*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [4]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [5]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [6]:
# TODO: Import any additional functionality you may need here
from sklearn import cross_validation

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    X_all, y_all, test_size = float(num_test)/float(num_train+num_test), random_state=0)
# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: ** The three models I am going to run are: SVM, Gaussian Naive Bayes and Ensemble Method with AdaBoost.

**SVM**

A real world example of using SVM is trying to classify stocks into winners and losers based on characteristics of the stock. (http://content.iospress.com/download/algorithmic-finance/af016?id=algorithmic-finance%2Faf016). You can take the 10% best and worst performing stocks over a certain time period and then run an SVM classifier to build the parameters. Then you could test this model in an out of sample data set and invest accordingly.
A strength of SVM is that they work well in complicated domains when there is a clear separator in the classification. Therefore, they perform well on data sets that aren't very noisy.
A weakness of SVM's is that the training time increases rapidly as it can be squared or cubic. Therefore, SVM's are slow and don't work well when the data set is quite large with lots of features.
Given that the dataset we are looking at is not very complicated, there aren't too many data points, an SVM should work well. Also, it is imaginable that there will be a clear separation between the classes given that passing and failing should not be very noisy. Doing a lot of homework and failing would be quite rare.

**Gaussian Naive Bayes **

A real world example of using Gaussian Naive Bayes is taking fMRI images and trying to determine if the patient was looking at one image type or the other, for example a hammer or a building (http://www.cs.cmu.edu/~tom/10601_sp09/lectures/NBayes2_2-2-2009-ann.pdf). As the feature variable is continuous, location of activity on a scan, the Gaussian Naive Bayes implementation normalizes the feature data to compute the classification.
The strength of Gaussian Naive Bayes is that the time to implement is quite low and multiple features can be handled easily. It will perform well when you don't have much data as it does not require much training data to estimate the classification parameters.
A weakness of Gaussian Naive Bayes is that the model assumes complete independance amongst the feautures. Therefore, the model will not perform well when the features are correlated, such as the housing data. The size, number of bedrooms, number of bathrooms in a house are likely correlated.
I think this model could be a good candidate as there are not a lot of training points, and yet many features. The Gaussian Naive Bayes will be able to handle this well. My concern is the independance assumption as there could be a correlation between absences, studytime and internt potentially.

**Ensemble Methods with AdaBoost**

A real world example of AdaBoost is detecting faces by investigating the pixels that have the strongest prediction (https://www.vision.caltech.edu/html-files/EE148-2005-Spring/pprs/viola04ijcv.pdf).
The strength of AdaBoost, and Boosting in general, is that it will find the features that do the best in classifying and ignore features that are just noise. Therefore, boosting will work well when there are features that strongly correlate to the different classes.
A weakness of AdaBoost is that it tries very hard to classify correctly. If a data point is mis-labelled for instance, the model will try to label it correctly and impair the out of sample testing. Therefore, boosting does not work particularly well with noisy, mislabelled data (https://en.wikipedia.org/wiki/Boosting_(machine_learning)#Criticism).
I think boosting will do a good job with our data because there are some strong features that should predict passing or failing, such as study time, activities and absences. Also, the data should be correctly labelled so I don't have concerns with too noisy of data.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [7]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [11]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier

# TODO: Initialize the three models
clf_A = svm.SVC(random_state=1)
clf_B = GaussianNB()
clf_C = AdaBoostClassifier(random_state=1)

# TODO: Set up the training set sizes
import random
random_100 = random.sample(range(0,300),100)
random_200 = random.sample(range(0,300),200)
X_train_100 = X_train.iloc[random_100]
y_train_100 = y_train.iloc[random_100]

X_train_200 = X_train.iloc[random_200]
y_train_200 = y_train.iloc[random_200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

#little shorter than previous code:
for clf in [clf_A, clf_B, clf_C]:
    train_predict(clf,X_train_100, y_train_100, X_test, y_test)
    train_predict(clf,X_train_200, y_train_200, X_test, y_test)
    train_predict(clf,X_train_300, y_train_300, X_test, y_test)
    
# OLD CODE:
#Support Vector Machine Model:
#train_predict(clf_A,X_train_100, y_train_100, X_test, y_test)
#train_predict(clf_A,X_train_200, y_train_200, X_test, y_test)
#train_predict(clf_A,X_train_300, y_train_300, X_test, y_test)

#Gaussian Naive Bayes Model:
#train_predict(clf_B,X_train_100, y_train_100, X_test, y_test)
#train_predict(clf_B,X_train_200, y_train_200, X_test, y_test)
#train_predict(clf_B,X_train_300, y_train_300, X_test, y_test)

#AdaBoost Boosting Model:
#train_predict(clf_C,X_train_100, y_train_100, X_test, y_test)
#train_predict(clf_C,X_train_200, y_train_200, X_test, y_test)
#train_predict(clf_C,X_train_300, y_train_300, X_test, y_test)


Training a SVC using a training set size of 100. . .
Trained model in 0.0045 seconds
Made predictions in 0.0015 seconds.
F1 score for training set: 0.8554.
Made predictions in 0.0014 seconds.
F1 score for test set: 0.7712.
Training a SVC using a training set size of 200. . .
Trained model in 0.0069 seconds
Made predictions in 0.0040 seconds.
F1 score for training set: 0.8889.
Made predictions in 0.0023 seconds.
F1 score for test set: 0.7770.
Training a SVC using a training set size of 300. . .
Trained model in 0.0120 seconds
Made predictions in 0.0085 seconds.
F1 score for training set: 0.8692.
Made predictions in 0.0031 seconds.
F1 score for test set: 0.7586.
Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0017 seconds
Made predictions in 0.0009 seconds.
F1 score for training set: 0.8696.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.7634.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0018 seconds
Made pred

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

**Note to grader: I randomized the selection for the training points so what is printed above may vary slightly from what is in the table below. The table consists of the values from the first time I ran all the models **

** Classifer 1 - Support Vector Machine Model**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.3855 seconds           |  0.0020 seconds        |  0.8535          | 0.7763          |
| 200               |0.0068 seconds           |  0.0020 seconds        |  0.8742          | 0.7534          |
| 300               |0.0121 seconds           |  0.0027 seconds        |  0.8692          | 0.7586          |

** Classifer 2 - Gaussian Naive Bayes Model**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.0014 seconds           |  0.0005 seconds        |  0.3038          | 0.3291          |
| 200               |0.0015 seconds           |  0.0005 seconds        |  0.8015          | 0.7612          |
| 300               |0.0017 seconds           |  0.0006 seconds        |  0.8088          | 0.7500          |

** Classifer 1 - Ensemble Boosting Model with AdaBoost**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.3125 seconds           |  0.0105 seconds        |  0.9778          | 0.7167          |
| 200               |0.2388 seconds           |  0.0109 seconds        |  0.8919          | 0.7910          |
| 300               |0.2963 seconds           |  0.0120 seconds        |  0.8688          | 0.7794          |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** The best model for determining when to intervene with Students is a Support Vector Machine. When picking a model, there are two important features to consider - does it generalize well and is it efficient to implement. The Support Vector Machine was the most reliable in our testing of working out of sample. Out of sample testing gives us the reassurance that we didn't build a model that only works with certain data. While some other models performed well out of sample, their performance was more variable, suggesting some potential overfitting.

Secondly, the Support Vector Machine was very quick in prediction. While it wasn't the fastest prediction time, it was the best combination of prediction time and performance out of sample. Prediction time is important because we may have limited resources in running this model frequently. One caveat, this model doesn't necessarily scale well with huge datasets. If we were going to roll this out across the country with millions of students, we may need to review this choice.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** The chosen model, a Support Vector Machine, attempts to classify students based of a number of attributes. The model works by determining the best dividing line between the two groups. This dividing line is optimized by trying to maximize the number of correct predictions, even if the line is moved around. This concept is called margin, we want to maximize margin.

To build the model, we looked at smaller subsets of the student population and maximized the margin on these small subsets to come up with parameters that did a good job classifying the students. We then used these parameters to classify the students who we did not look at in training, the out of sample students. On the various populations I trained the model on, the models classified about 86% of the students correctly. On the out of sample populations, the model classified about 76% correctly.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [12]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn import grid_search
from sklearn.metrics import make_scorer, f1_score

# TODO: Create the parameters list you wish to tune
parameters = {'kernel':('linear','rbf','poly','sigmoid'), 'C':[0.0001,0.0005,0.001,0.005,0.01,0.05,0.1], 'gamma':[0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1]}

# TODO: Initialize the classifier
clf = svm.SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))
print(grid_obj.best_estimator_)

Made predictions in 0.0049 seconds.
Tuned model has a training F1 score of 0.8344.
Made predictions in 0.0017 seconds.
Tuned model has a testing F1 score of 0.8000.
SVC(C=0.05, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The model using Grid Search, varying the parameters for kernel, C and gamma, produced F1 score of 0.8344 for the training set and 0.8000 for the test set.

Compared to the untuned model, the F1 score is a bit higher for the test set, although it is lower for the training set. The untuned training F1 score was 0.85 to 0.87 so it decreased slightly while the untuned test F1 was 0.75 to 0.77 so that increased slightly.

By printing out the best estimator, I can see that the linear model was chosen with a C value of 0.05 and a gamma value of 0.0001. However, since gamma is not used in a linear model, only when we do the kernel trick, this parameter is irrelevent. Therefore, a linear model with a relatively low C value did best on the test set. The low value of C means that there is a softer margin during training, meaning that misclassifying a few points is ok if the margin is a bit wider.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.